In [0]:
import os
if str.lower(dbutils.widgets.getAll().get("install_ml_libs", "False")) == "true":
  print("Installing ML libraries...")
  os.system("pip install -qqqq lightgbm databricks-feature-store bayesian-optimization mlflow hyperopt shap databricks-automl-runtime")
  dbutils.library.restartPython()

In [0]:
import json
import os

def _get_config():

  # Config should be stored at <solution_accelerator_root>/notebooks/demo_setup/config.json
  # We are expecting _init.ipynb to be run from somewhere within the notebooks folder
  # Databricks will set the current working directory to currently running notebook
  # This will traverse up the folder structure until it finds the notebooks folder
  notebook_root = os.getcwd()
  while not notebook_root.endswith('/notebooks'):
      parent = os.path.dirname(notebook_root)
      if parent == notebook_root:
          break
      notebook_root = parent
  notebook_root
  json_config_path = os.path.join(notebook_root, "demo_setup/config.json")

  # The DABS setup job should inject the config as job params, and set the load_config_from_file param to False
  # If this check is absent, we'll assume the user is running the notebook interactively and load the config from the file
  config = dbutils.widgets.getAll()
  if str.lower(config.get("load_config_from_file", "True")) == "true":
    try:
      with open(json_config_path, "r") as f:
        config = json.load(f)
    except:
      # If we can't load the config from the file, assume that the setup process wasn't run
      print(f"❌ Could not load config from {json_config_path}")
      print(f"Make sure that you have:")
      print(f"   1. Deployed this Solution Accelerator using Databricks Asset Bundles from the UI")
      print(f"   2. Run the deploy-iops-demo-workflow job, which will create the config.json file")
      print(f"If you are deploying the bundle from the CLI, you will need to manually configure the config.json file")
      print(f"Expected paramters are:")
      print(f"   1. catalog_name")
      print(f"   2. schema_name")
      print(f"   3. volume_name")
      raise Exception(
        "Could not load config. Please ensure you have run the setup job (deploy-iops-demo-workflow) created by Databricks Asset Bundles")
  else:
    # Preserve existing keys if config file already exists
    existing_config = {}
    if os.path.exists(json_config_path):
      try:
        with open(json_config_path, "r") as f:
          existing_config = json.load(f)
        print(f"📄 Found existing config file, preserving additional keys...")
      except:
        print(f"⚠️ Could not read existing config file, creating new one...")
        existing_config = {}
    
    # Merge widget config with existing config (widget config takes precedence for standard keys)
    merged_config = existing_config.copy()
    merged_config.update(config)
    
    with open(json_config_path, "w") as f:
      json.dump(merged_config, f, indent=2)
    
    # Update config to use the merged version
    config = merged_config

  return config

config = _get_config()

catalog_name = config['catalog_name']
schema_name = config['schema_name']
volume_name = config['volume_name']
genie_id = config.get('genie_id')
dashboard_id = config.get('dashboard_id')
app_url = config.get('app_url')

print(f"✅ Variables configured:")
print(f"   📁 Catalog: {catalog_name}")
print(f"   📂 Schema: {schema_name}")
print(f"   💾 Volume: {volume_name}")

In [0]:
### Helper functions for the demo ###

In [0]:
def drop_fs_table(table_name):
  from databricks.feature_store import FeatureStoreClient
  fs = FeatureStoreClient()
  try:
    fs.drop_table(table_name)  
  except Exception as e:
    print(f"Can't drop the fs table, probably doesn't exist? {e}")
  try:
    spark.sql(f"DROP TABLE IF EXISTS `{table_name}`")
  except Exception as e:
    print(f"Can't drop the delta table, probably doesn't exist? {e}")

In [0]:

def display_launch(url, text, description=None):
    displayHTML(f"""
        <div style="padding: 20px; border-left: 5px solid #ff6b1a; background-color: #fff8f0; margin: 15px 0; border-radius: 8px;">
            <h3 style="margin: 0 0 15px 0; color: #ff6b1a; display: flex; align-items: center;">
                {"📱 " + description if description else ""}
            </h3>
                <p style="margin: 10px 0 0 0;">
                    <a href='{url}' target='_blank' 
                       style='background-color: #ff6b1a; color: white; padding: 12px 24px; text-decoration: none; 
                              border-radius: 5px; font-weight: bold; display: inline-block; font-size: 16px;'>
                       🚀 {text} →
                    </a>
                </p>
        </div>
        """)

In [0]:
def display_launch_error(message, detail):
    displayHTML(f"""
    <div style="padding: 20px; border-left: 5px solid #dc3545; background-color: #fff5f5; margin: 15px 0; border-radius: 8px;">
        <h3 style="margin: 0 0 10px 0; color: #dc3545;">⚠️ {message}</h3>
        <p style="margin: 5px 0; font-size: 16px;">
            {detail}
        </p>
    </div>
    """)